# Relating $p_F$ and $\alpha$

## The Linear Case

This is already done for the linear case.

$$p_F = 1 - \Phi(\alpha  / |a|)$$

## The Brownian Case

### The Problem

In the Brownian case where $g(x) = \max_k (B x)_k$, we know that

$$p_F = 1 - \Phi_{\vec{0}, B B^T} (\vec{\alpha})$$

where $\Phi_{\vec{0}, B B^T}$ is the multivariate *joint* CDF with mean 0 and covariance $B B^T$ and $\vec{\alpha} = (\alpha)_{i=1}^K$. The joint multivariate CDF takes in a vector $\vec{v}$ and calculates the probability that $X \sim \mathcal{N}(\vec{0}, B B^T)$ has any component $X_k > v_k$. The multivariate joint CDF takes a long time to compute, especially in high dimensions.
### A new Brownian loss

Let's let $\{ X_i \sim \mathcal{N}(0, \sigma_i^2)\}_{i=1}^k$ be independent random variables. Define 
$$g(x) = \max_{i} x_i.$$
Then
$$p_F = 1 - \prod_{i=1}^k \Phi((\alpha) / \sigma_i).$$
Since $p_F$ will be very small, we will use $\log$ to maintain numerical stability. We have

$$\log (1 - p_F) = \sum_{i=1}^k \log \Phi((\alpha) / \sigma_i)$$

so we can solve for $\alpha$ using Newton's method. Also, since $\log ( 1 - p_F) \approx - p_F$, we can instead solve for
$$p_F = - \sum_{i=1}^k \log \Phi(\alpha / \sigma_i).$$

The derivative can be simplified as well since $\Phi(\alpha / \sigma_i)$ will be very close to one for small $p_F$, and $\log \Phi(\alpha / \sigma_i)$ will be about 0.

$$- \frac{d}{d \alpha} \sum_{i=1}^k \log \Phi(\alpha / \sigma_i) = \sum_{i=1}^k \frac{\rho(\alpha / \sigma_i)}{\sigma_i \Phi(\alpha/ \sigma_i)} = \sum_{i=1}^k e^{\log \rho(\alpha / \sigma_i) - \log \Phi(\alpha / \sigma_i)} / \sigma_i \approx \sum_{i=1}^k \frac{\rho(\alpha / \sigma_i)}{\sigma_i}$$

### New Brownian Implementation

Let's implement this new loss function, and show that the approximate relation between $p_F$ and $\alpha$ is very accurate.

In [8]:
import numpy as np
from loss import *

In [9]:
class NewBrownian(Loss):
    def __init__(self, n : int):
        self.b = [np.random.uniform() for i in range(0, n)]
        self.n = n
    def compute(self, x : np.array) -> float:
        x = list(x)
        return max(list(map(
            lambda i : self.b[i] * x[i],
            range(0, self.n)
        )))
    def __str__(self):
        return "New Brownian"

In [10]:
nb = NewBrownian(3)

In [11]:
nb.b

[0.6746852326701887, 0.6942067287999607, 0.8941426802466219]

In [12]:
nb.compute(np.array([1,2,3]))

2.6824280407398655

In [14]:
from scipy.stats import norm
from math import log

In [22]:
def brown_pF(g : NewBrownian, alpha: float) -> float:
    # only works for small p_F
    return -sum(list(map(
        lambda sigma_i: log(norm.cdf(alpha / sigma_i)),
        g.b
    )))

In [24]:
brown_pF(nb, 5)

1.1227116743798745e-08

In [25]:
from scipy import optimize

In [32]:
def brown_alpha(g: NewBrownian, pF: float) -> float:
    
    # only works for small p_F
    
    def fprime(alpha): 
        return -sum(list(map(
            lambda sigma_i : 
                norm.pdf(alpha/sigma_i) / sigma_i,
            g.b
        )))

    def f(alpha):
        return (brown_pF(g, alpha) - pF)
    
    return optimize.newton(f, 0, fprime)

In [33]:
brown_alpha(nb, 1.1227116743798745e-08)

5.000000000096113